In [418]:
#This is a starting point
#uncomment below to install merlin library
#!pip install merlin, nvtabular, merlin.models
#I had to downgrade keras to 12.2.0 there maybe issues regarding tensorflow in the future
#run lines below if you have also have an error ab not finding a keras package

#!pip uninstall keras
#!pip install keras==2.12.0


#These are the same imports from
#https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb

import os
import pandas as pd
import nvtabular as nvt
import matplotlib.pyplot as plt

from merlin.models.utils.example_utils import workflow_fit_transform
import merlin.io
import tensorflow as tf

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
from nvtabular.ops import *
from merlin.core.utils import download_file
from merlin.schema.tags import Tags

# Load and split

In [386]:
df = pd.read_csv('/Users/andrew/Desktop/projects/recsys_data/2023-10-05 9_23pm.csv').dropna()
df = df.drop(columns = ['CLICKSTREAM_EVENTS_TOTAL.1'])
#  ^ This needs point to your local .csv too big for github

In [387]:
#accidentally added an extra CLICKSTREAM_EVENTS_TOTAL column
df['TARGET'] = 0
df.loc[df['EVENT_NAME'] == 'order', 'TARGET'] = 1

In [432]:
df['TARGET'].sum()/len(df['TARGET'])

0.03777583047877001

In [389]:
#num rows
data_size = df.shape[0]

#We're doing a 20/80 train/validate split change the .2 to change the split
train_split_ratio = int(.2 * data_size)

train_data = df[train_split_ratio:].to_parquet("train_data.parquet")
valid_data = df[:train_split_ratio].to_parquet("valid_data.parquet")

train = Dataset("train_data.parquet")
valid = Dataset("valid_data.parquet")

/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/parquet.py:343: UserWarning: Row group memory size (1394257212) (bytes) of parquet file is bigger than requested part_size (1073741824) for the NVTabular dataset.A row group memory size of 128 MB is generally recommended. You can find info on how to set the row group size of parquet files in https://nvidia-merlin.github.io/NVTabular/main/resources/troubleshooting.html#setting-the-row-group-size-for-the-parquet-files
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


# EDA

# NVTabular

In [390]:
#This is how to iniate a dataset using NVtabular an NVIDIA 
#optimized ETL/feature engineering(i think?) library
#data = nvt.io.dataset.Dataset('data/reviews.csv')

In [391]:
# df.columns

In [392]:
categorical_user_features = ['COUNTRY', 'DERIVED_GENDER_BY_NAME', 'EVENT_NAME']
categorical_item_features = ['STYLE', 'TAXONOMY_STYLE', 'COLOR_NAME', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS', 'TEAM', 'FRANCHISE', 'PRODUCT_GROUP']

In [393]:
#Features needed to Transform still: FIRST_PURCHASE_AT, FIRST_VISIT_AT, LATEST_VISIT_AT
# LATEST_PURCHASE_AT, EVENT_TIMESTAMP

In [394]:
#DON'T FORGET TARGET ENCODING CATEGORICAL VALUES

In [395]:
user_id = ["USER_ID"] >> Categorify(dtype = "int32") >> TagAsUserID()
item_id = ["ITEM_ID"] >> Categorify(dtype = "int32") >> TagAsItemID()
item_features = categorical_item_features >> Categorify(dtype = "int32") >> TagAsItemFeatures()
user_features = categorical_user_features >> Categorify(dtype = "int32") >> TagAsUserFeatures()

In [396]:
#Make this more succinct
CLICKSTREAM_EVENTS_TOTAL = (
    ['CLICKSTREAM_EVENTS_TOTAL']
    >>FillMissing(0)
    >>LogOp()
    >>Normalize()
    >>LambdaOp(lambda col: col.astype("float32"))
    >>TagAsUserFeatures()
)

#has the price changed over time?
PRICE_INFORMATION = (
    ['PRICE_INFORMATION']
    >>FillMissing(0)
    >>LogOp()
    >>Normalize()
    >>LambdaOp(lambda col: col.astype("float32"))
    >>TagAsItemFeatures()
)

AVG_REVIEW_SCORE = (
    ['AVG_REVIEW_SCORE']
    >>FillMissing(0)
    >>LogOp()
    >>Normalize()
    >>LambdaOp(lambda col: col.astype("float32"))
    >>TagAsItemFeatures()
)

In [397]:
categorical_features = (categorical_item_features) #Just a place holder... redo with better categories
label = nvt.ColumnSelector(['TARGET'])
te_features = categorical_features >> TargetEncoding(label)
te_features_norm = te_features >> Normalize() >> LambdaOp(lambda col: col.astype('float32')) >> TagAsItemFeatures()

In [398]:
target = (
    ['TARGET'] >> AddMetadata(tags=[Tags.BINARY_CLASSIFICATION, "TARGET"])
)

In [420]:
outputs = (
    user_id + 
    target + 
    CLICKSTREAM_EVENTS_TOTAL + 
    item_id +
    te_features_norm +
    PRICE_INFORMATION +
    AVG_REVIEW_SCORE
)

workflow = nvt.Workflow(outputs)

In [425]:
%%time

train_data = Dataset('train_data.parquet') #Direct to your local files
valid_data = Dataset('valid_data.parquet')

workflow.fit(train_data)

workflow.transform(train_data).to_parquet(
    'train_data_transformed' #Feel free to replace with your own local file path
)
workflow.transform(valid_data).to_parquet(
    'valid_data_transformed' #same here
)

/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/parquet.py:343: UserWarning: Row group memory size (1394257212) (bytes) of parquet file is bigger than requested part_size (1073741824) for the NVTabular dataset.A row group memory size of 128 MB is generally recommended. You can find info on how to set the row group size of parquet files in https://nvidia-merlin.github.io/NVTabular/main/resources/troubleshooting.html#setting-the-row-group-size-for-the-parquet-files
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/pandas/co

CPU times: user 18 s, sys: 4.07 s, total: 22 s
Wall time: 22.4 s


/Users/andrew/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Transform Timestamped data

In [426]:
train = Dataset('train_data_transformed', engine = 'parquet')
valid = Dataset('valid_data_transformed', engine = 'parquet')

/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [427]:
schema = train.schema
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]

In [429]:
train['TARGET']

TypeError: 'Dataset' object is not subscriptable

In [413]:
model = mm.DLRMModel(
    schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryOutput(target_column)
)

model.compile(optimizer="adagrad", metrics = [tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=1024, epochs = 1, learning_rate = 0.001)

TypeError: Loader.__init__() got an unexpected keyword argument 'learning_rate'

In [365]:
metrics = model.evaluate(valid, batch_size = 1024, return_dict = True)

298/298 [==============================] - 1s 3ms/step - loss: nan - auc_1: 0.5000 - regularization_loss: 0.0000e+00 - loss_batch: nan


In [ ]:
metrics

# Model

In [ ]:
train

In [ ]:
model = mm.DLRMModel(
    train_transformed.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(train_transformed.schema)
)

model.compile(optimizer="adagrad", run_eagerly=False)
model.fit(train_transformed, validation_data=valid_transformed, batch_size=1024)
eval_metrics = model.evaluate(valid_transformed, batch_size=1024, return_dict=True)